# **Connecting to Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Mapping Image-IDs to Labels**

In [2]:
import pandas as pd

# Load the labels.csv file
labels_df = pd.read_csv('/content/drive/MyDrive/labels.csv')

# Create a dictionary that maps image IDs to their corresponding labels
labels_dict = {}
for index, row in labels_df.iterrows():
    labels_dict[row['image_id']] = row['Object']

# **Split data by train_test_split**

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
train_image_ids, test_image_ids, train_labels, test_labels = train_test_split(list(labels_dict.keys()),list(labels_dict.values()), test_size=0.2, random_state=42)

# **Load the Images**

In [5]:
import cv2
import numpy as np

In [6]:
image_size = (224, 224)
train_images = []
test_images = []
for image_id in train_image_ids:
    image_path = f'/content/drive/MyDrive/Archaeological_Objects_Cropped/{image_id}.jpg'
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    train_images.append(image)

for image_id in test_image_ids:
    image_path = f'/content/drive/MyDrive/Archaeological_Objects_Cropped/{image_id}.jpg'
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    test_images.append(image)

# **Convert the lists to NumPy arrays**

In [7]:
train_images = np.array(train_images)
test_images = np.array(test_images)

# **Defining a backbone Model**

*A backbone model is typically a convolutional neural network (CNN) that has been pre-trained on a large dataset, such as ImageNet, to learn general features that can be applied to a wide range of tasks. The pre-trained model is then fine-tuned on a smaller dataset specific to the task at hand, such as object detection, segmentation, or classification.*

In [8]:
from keras.applications import VGG16
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.regularizers import l2
from keras.models import Model
backbone = VGG16(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# **Define the RPN**

*RPN is a neural network that generates region proposals, which are regions of interest in an image that may contain objects. The RPN takes an image as input and outputs a set of bounding boxes, called anchors, that are likely to contain objects. These anchors are then used as input to a classifier and regressor to refine the object detection process.*

In [9]:
rpn = Conv2D(512, (3, 3), activation='relu', padding='same',  kernel_regularizer=l2(0.01))(backbone.output)
rpn = Conv2D(512, (3, 3), activation='relu', padding='same',  kernel_regularizer=l2(0.01))(rpn)
rpn = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(rpn)

# **Define the classification layer**

In [10]:
classification = Flatten()(rpn)
classification = Dense(4096, activation='relu', kernel_regularizer=l2(0.01))(classification)
classification = Dropout(0.5)(classification)
classification = Dense(len(labels_dict), activation='softmax')(classification)

# **Define the faster R-CNN model**

*Faster R-CNN is a popular object detection algorithm that builds upon the success of R-CNN and its variants. It's a powerful tool for detecting objects in images and videos, and has been widely adopted in various applications, including self-driving cars, surveillance systems, and medical image analysis.*

In [11]:
model = Model(inputs=backbone.input, outputs=[rpn, classification])

# **Compile the model**

In [12]:
model.compile(loss=['mean_squared_error', 'categorical_crossentropy'], optimizer='adam', metrics=[['accuracy'] , ['accuracy']])

# **Train the model**

In [13]:
from keras.callbacks import EarlyStopping

In [ ]:
model.fit(train_images,
          [np.zeros((len(train_images), 7, 7, 512))],
          epochs=10,
          batch_size=32,
          validation_data=(test_images, [np.zeros((len(test_images), 7, 7, 512))]),
          callbacks=[EarlyStopping(patience=5)])

Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1120s 80s/step - conv2d_2_accuracy: 0.5076 - loss: 71.9931 - val_conv2d_2_accuracy: 1.0000 - val_loss: 14.0735
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1177s 82s/step - conv2d_2_accuracy: 0.9720 - loss: 13.7828 - val_conv2d_2_accuracy: 0.8822 - val_loss: 10.5664
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1168s 82s/step - conv2d_2_accuracy: 0.9318 - loss: 9.3447 - val_conv2d_2_accuracy: 0.8967 - val_loss: 7.0076
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1138s 82s/step - conv2d_2_accuracy: 0.9503 - loss: 6.5640 - val_conv2d_2_accuracy: 0.9592 - val_loss: 5.2817
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1134s 81s/step - conv2d_2_accuracy: 0.9615 - loss: 4.9698 - val_conv2d_2_accuracy: 0.8633 - val_loss: 4.1521
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1149s 81s/step - conv2d_2_accuracy: 0.9148 - loss: 3.9334 - val_conv2d_2_accuracy: 0.9471 - val_loss: 3.3418
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1187s 82s/step - conv2d_2_accuracy: 0.8846 - loss: 3.1833 - val_conv2d_2_acc

#**Save the model**

In [14]:
model.save('archeological_artifacts_model.keras')

#**Evaluate the model on the test data**

In [ ]:
test_loss, test_accuracy = model.evaluate(test_images, [np.zeros((len(test_images), 7, 7, 512))], verbose=0)
print(f'Test loss: {test_loss:.3f}')
print(f'Test accuracy: {test_accuracy:.3f}')

# **Mean Average Precision (mAP)**
It's a metric used to evaluate the performance of object detection models, particularly in computer vision tasks.

In object detection, the goal is to locate objects within an image and classify them into different categories (e.g., person, car, dog, etc.). The model predicts bounding boxes around the objects, along with a confidence score indicating how likely the object is to be present.

# **Calculation of mAP**

In [ ]:
import numpy as np
from sklearn.metrics import average_precision_score

def calculate_ap(y_true, y_pred):
    if np.sum(y_true) == 0:
        return 0
    ap = average_precision_score(y_true, y_pred)
    return ap

def calculate_map(y_true, y_pred, labels_dict):
    aps = []
    for i in range(len(labels_dict)):
        y_true_class = np.array([1 if label == i else 0 for label in y_true])
        y_pred_class = y_pred[:, i]
        ap = calculate_ap(y_true_class, y_pred_class)
        aps.append(ap)
    map = np.mean(aps)
    return map

# Make predictions on the test set
y_pred = model.predict(test_images)[1]

# Convert the predictions to a binary format (0 or 1) for each class
y_pred_binary = (y_pred > 0.5).astype(int)

# Define labels_dict
labels_dict = {i: f'class{i}' for i in range(y_pred.shape[1])}

# Calculate the mean average precision (mAP)
map = calculate_map(test_labels, y_pred_binary, labels_dict)
print(f'mAP: {map:.3f}')